To-do:
- dodělat zemi narození, ale možná až sólo datasetem?

Chcu:
- *ofiko název*
- *bio*
- x žánr
- x kapela/jednotlivec? (Ale bacha u interpret_hlavni!)
- *země původu (co když víc?)*
- x pohlaví
- x *etnicita?*
- x rok narození 
- x *místo narození*
- x rok úmrtí
- *místo úmrtí*
- x začátek kariéry
- x konec kariéry

Možná ještě:
- last.fm id
- x spotify id
- ocenění?

In [48]:
import os
import json
import requests

In [49]:
from wikidata.client import Client
import wikidata.entity

In [50]:
with open(os.path.join("wikidata","wiki_ids.json"), "r") as file:
    qecka = json.load(file)

In [51]:
def wikidates(id):
    client = Client()
    entity = client.get(id, load=True)
    return entity

In [52]:
def vsechny_atributy(q,p):
    atributy = []
    for x in wikidates(q).attributes['claims'][p]:
        atributy.append(x['mainsnak']['datavalue']['value']['id'])
    return(atributy)

In [53]:
def get_wikidata_item_label(wikidata_id, language):
    url = f'https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json'
    response = requests.get(url)
    data = response.json()
    label = data['entities'][wikidata_id]['labels'][language]['value']
    return label

## A jdem na to!

In [54]:
wikidates('Q43453').attributes['claims']['P17'][0]['mainsnak']['datavalue']['value']['id']

'Q213'

In [55]:
def stahni_wikidata(slovnik):
    
    slovnik = {k: v for k, v in slovnik.items() if v is not None}
    
    kody = {}
    vysledky = []
    chyby = []
    c = 0
    
    def wikidates(id):
        client = Client()
        entity = client.get(id, load=True)
        return entity
    
    def atributy(p):
        atributy = []
        for x in balicek.attributes['claims'][p]:
            atributy.append(x['mainsnak']['datavalue']['value']['id'])
        return(atributy)
    
    def rozkodovani(wikidata_id):
        url = f'https://www.wikidata.org/wiki/Special:EntityData/{wikidata_id}.json'
        response = requests.get(url)
        data = response.json()
        try:
            label = data['entities'][wikidata_id]['labels']['cs']['value']
        except:
            label = data['entities'][wikidata_id]['labels']['en']['value']
        kody[wikidata_id] = label
        return label        
    
    def rozkoduj(peckakvecka):
        if isinstance(peckakvecka, list):
            novy_seznam = []
            for x in peckakvecka:
                novy_seznam.append(rozkodovani(x))
            citelne = novy_seznam
        elif isinstance (peckakvecka, str):
            citelne = rozkodovani(peckakvecka)
        else:
            citelne = peckakvecka
        return citelne
        
    for key, value in slovnik.items():
        
        if value != None:
        
            interpret = {}
            interpret['interpret_hlavni'] = key
            interpret['id'] = value
            
            try:
            
                balicek = wikidates(value)
                
                interpret['stazeno'] = True

                try:
                    interpret['nazev_wiki'] = balicek.label.texts['en']
                except:
                    interpret['nazev_wiki'] = None

                try:
                    interpret['popis_en'] = balicek.description.texts['en']
                except:
                    interpret['popis_en'] = None

                try:
                    interpret['popis_cs'] = balicek.description.texts['cs']
                except:
                    interpret['popis_cs'] = None
                    
                try:
                    interpret['velikost'] = rozkoduj(balicek.attributes['claims']['P31'][0]['mainsnak']['datavalue']['value']['id'])
                except: 
                    interpret['velikost'] = None

                try:
                    interpret['zanry'] = rozkoduj(atributy('P136'))
                except:
                    interpret['zanry'] = None

                try:
                    interpret['datum_narozeni'] = balicek.attributes['claims']['P569'][0]['mainsnak']['datavalue']['value']['time']
                except:
                    interpret['datum_narozeni'] = None

                try:
                    interpret['datum_umrti'] = balicek.attributes['claims']['P570'][0]['mainsnak']['datavalue']['value']['time']
                except:
                    interpret['datum_umrti'] = None

                try:
                    interpret['zacatek_pusobeni'] = balicek.attributes['claims']['P2031'][0]['mainsnak']['datavalue']['value']['time']
                except:
                    interpret['zacatek_pusobeni'] = None

                try:
                    interpret['konec_pusobeni'] = balicek.attributes['claims']['P2032'][0]['mainsnak']['datavalue']['value']['time']
                except:
                    interpret['konec_pusobeni'] = None

                try:
                    interpret['zalozeni'] = balicek.attributes['claims']['P571'][0]['mainsnak']['datavalue']['value']['time']
                except:
                    interpret['zalozeni'] = None

                try:
                    interpret['zanik'] = balicek.attributes['claims']['P576'][-1]['mainsnak']['datavalue']['value']['time']
                except:
                    interpret['zanik'] = None
                    
                try:
                    interpret['gender'] = rozkoduj(balicek.attributes['claims']['P21'][0]['mainsnak']['datavalue']['value']['id'])
                except:
                    interpret['gender'] = None
                
                try:
                    try:
                        odkud = atributy('P19')[-1]
                    except:
                        odkud = atributy('P740')[-1]
                    interpret['misto_puvodu'] = rozkoduj(odkud)
                    try:
                        interpret['zeme_puvodu'] = rozkoduj(wikidates(odkud).attributes['claims']['P17'][0]['mainsnak']['datavalue']['value']['id'])
                    except:
                        interpret['zeme_puvodu'] = None
                except:
                    interpret['misto_puvodu'] = None
                    interpret['zeme_puvodu'] = None
                    
                try:
                    interpret['obcanstvi'] = rozkoduj(atributy('P27'))
                except:
                    interpret['obcanstvi'] = None

                try:
                    interpret['etnicita'] = rozkoduj(atributy('P172'))
                except:
                    interpret['etnicita'] = None

                try:
                    interpret['hlas'] = rozkoduj(atributy('P412'))
                except:
                    interpret['hlas'] = None

                try:
                    interpret['nastroje'] = rozkoduj(atributy('P1303'))
                except:
                    interpret['nastroje'] = None
                    
                try:
                    interpret['oceneni'] = rozkoduj(atributy('P166'))
                except:
                    interpret['oceneni'] = None

                try:
                    interpret['nabozenstvi'] = rozkoduj(atributy('P140'))
                except:
                    interpret['nabozenstvi'] = None

                try:
                    interpret['orientace'] = rozkoduj(atributy('P91'))
                except:
                    interpret['orientace'] = None

                try:
                    interpret['spotify_id'] = balicek.attributes['claims']['P1902'][0]['mainsnak']['datavalue']['value']
                except:
                    interpret['spotify_id'] = None

                try:
                    interpret['spotify_odberatele'] = balicek.attributes['claims']['P1902'][0]['qualifiers']['P3744'][0]['datavalue']['value']['amount']
                except:
                    interpret['spotify_odberatele'] = None
                
            except:
                
                interpret['stazeno'] = False
                chyby.append(key)
                
            c = c + 1
            
            print(f'{c} / {len(slovnik)} / {key}')
            
            vysledky.append(interpret)
            
        else:
            pass
    
    if len(chyby) > 0:
        print(f"""Chyby: {(", ").join(chyby)}""")
    
    return(vysledky)

In [56]:
wikidates('Q6523170').attributes['claims']['P17'][0]['mainsnak']['datavalue']['value']['id']

'Q30'

In [57]:
import itertools

In [58]:
pokus = { "Ed Sheeran": "Q47447", "Lucie Bílá": "Q468282", "Kabát": "Q938649", "Lady Gaga": "Q19848", "Pharrell Williams": "Q14313", "Michal Hrůza": "Q12036727", "Karel Gott": "Q333187", "Szabo Peter": "Szabo Peter"}

In [61]:
pokusne_vysledky = stahni_wikidata(pokus)

1 / 8 / Ed Sheeran
2 / 8 / Lucie Bílá
3 / 8 / Kabát
4 / 8 / Lady Gaga
5 / 8 / Pharrell Williams
6 / 8 / Michal Hrůza
7 / 8 / Karel Gott
8 / 8 / Szabo Peter
Chyby: Szabo Peter


In [63]:
pokusne_vysledky

[{'interpret_hlavni': 'Ed Sheeran',
  'id': 'Q47447',
  'stazeno': True,
  'nazev_wiki': 'Ed Sheeran',
  'popis_en': 'English singer-songwriter',
  'popis_cs': 'anglický zpěvák a textař',
  'velikost': 'člověk',
  'zanry': ['lidová hudba', 'folk pop', 'pop music', 'folk', 'hip hop', 'rap'],
  'datum_narozeni': '+1991-02-17T00:00:00Z',
  'datum_umrti': None,
  'zacatek_pusobeni': '+2004-00-00T00:00:00Z',
  'konec_pusobeni': None,
  'zalozeni': None,
  'zanik': None,
  'gender': 'muž',
  'misto_puvodu': 'Halifax',
  'zeme_puvodu': 'Spojené království',
  'obcanstvi': ['Spojené království'],
  'etnicita': None,
  'hlas': ['tenor'],
  'nastroje': ['kytara', 'hlas', 'live looper'],
  'oceneni': ['Cena Grammy za nejlepší popový sólový výkon',
   'Favorite Pop/Rock Male Artist',
   'Brit Award for British Breakthrough Act',
   'Brit Award for British Male Solo Artist',
   'Brit Award for British Male Solo Artist',
   'Brit Award for British Album of the Year',
   'BRIT Awards',
   'BRIT Award

In [64]:
import pandas as pd

In [65]:
final = stahni_wikidata(qecka)

1 / 1152 / Michal Hrůza
2 / 1152 / Chinaski
3 / 1152 / P!Nk
4 / 1152 / Coldplay
5 / 1152 / Miro Žbirka
6 / 1152 / Support Lesbiens
7 / 1152 / Ewa Farna
8 / 1152 / David Guetta
9 / 1152 / Kabát
10 / 1152 / No Name
11 / 1152 / Lady Gaga
12 / 1152 / Lucie Bílá
13 / 1152 / Maroon 5
14 / 1152 / Lucie Vondráčková
15 / 1152 / Marek Ztracený
16 / 1152 / Divokej Bill
17 / 1152 / Xindl X
18 / 1152 / Onerepublic
19 / 1152 / Tomáš Klus
20 / 1152 / Rihanna
21 / 1152 / Mandrage
22 / 1152 / Avril Lavigne
23 / 1152 / Katy Perry
24 / 1152 / Linkin Park
25 / 1152 / Jason Derulo
26 / 1152 / Avicii
27 / 1152 / Enrique Iglesias
28 / 1152 / Mariah Carey
29 / 1152 / Eminem
30 / 1152 / David Deyl
31 / 1152 / Taylor Swift
32 / 1152 / Karel Gott
33 / 1152 / Calvin Harris
34 / 1152 / Dan Bárta
35 / 1152 / Václav Neckář
36 / 1152 / Miley Cyrus
37 / 1152 / Ed Sheeran
38 / 1152 / Olga Lounová
39 / 1152 / Sunrise Avenue
40 / 1152 / Green Day
41 / 1152 / Imagine Dragons
42 / 1152 / Janek Ledecký
43 / 1152 / Ellie Gou

341 / 1152 / Zayn
342 / 1152 / R.E.M.
343 / 1152 / Camila Cabello
344 / 1152 / Shaggy
345 / 1152 / Purple Disco Machine
346 / 1152 / Chris Brown
347 / 1152 / Zoe Wees
348 / 1152 / Shayne Ward
349 / 1152 / 50 Cent
350 / 1152 / Čechomor
351 / 1152 / Afrojack
352 / 1152 / Sto Zvířat
353 / 1152 / 24Kgoldn
354 / 1152 / Snoop Dogg
355 / 1152 / Carly Rae Jepsen
356 / 1152 / Biffy Clyro
357 / 1152 / Markéta Konvičková
358 / 1152 / Hana Hegerová
359 / 1152 / Tokio Hotel
360 / 1152 / Marta Jandová
361 / 1152 / Beatles
362 / 1152 / Basto!
363 / 1152 / Tom Grennan
364 / 1152 / Tim Berg
365 / 1152 / Timmy Trumpet
366 / 1152 / Jan Kalousek
367 / 1152 / Monika Absolonová
368 / 1152 / Basshunter
369 / 1152 / Prago Union
370 / 1152 / Big Ali
371 / 1152 / Michael Schulte
372 / 1152 / Michael Jackson
373 / 1152 / Mike Shinoda
374 / 1152 / Bts
375 / 1152 / Icona Pop
376 / 1152 / Blur
377 / 1152 / Imanbek
378 / 1152 / Banners
379 / 1152 / Imany
380 / 1152 / Mimi Webb
381 / 1152 / Buty
382 / 1152 / Melanie 

681 / 1152 / Cian Ducrot
682 / 1152 / Accept
683 / 1152 / Adam Levine
684 / 1152 / Ckay
685 / 1152 / Citizens!
686 / 1152 / Ciara
687 / 1152 / Cheat Codes
688 / 1152 / David Gilmour
689 / 1152 / Alice Cooper
690 / 1152 / Carlie Hanson
691 / 1152 / Cardi B
692 / 1152 / Štefan Margita
693 / 1152 / Renforshort
694 / 1152 / Redfoo
695 / 1152 / Sinplus
696 / 1152 / Aleš Háma
697 / 1152 / Aleš Brichta
698 / 1152 / Alex Da Kid
699 / 1152 / Xiii. Století
700 / 1152 / Delerium
701 / 1152 / Alexander Cardinale
702 / 1152 / Shinedown
703 / 1152 / Shontelle
704 / 1152 / Sasha Alex Sloan
705 / 1152 / Zdenka Tichotová
706 / 1152 / Zrní
707 / 1152 / Roger Waters
708 / 1152 / Danity Kane
709 / 1152 / Rolling Stones
710 / 1152 / Roman Dragoun
711 / 1152 / Rixton
712 / 1152 / Roman Holý
713 / 1152 / Zucchero
714 / 1152 / Zbyněk Drda
715 / 1152 / Dash Berlin
716 / 1152 / David Bowie
717 / 1152 / 5Angels
718 / 1152 / Dave Ramone
719 / 1152 / Daughtry
720 / 1152 / Dasha
721 / 1152 / Danzel
722 / 1152 / Dag

1008 / 1152 / Cocotte Minute
1009 / 1152 / Drake
1010 / 1152 / Lisa
1011 / 1152 / Fiedlerski
1012 / 1152 / Kryštof
1013 / 1152 / Y2K
1014 / 1152 / Topic
1015 / 1152 / Script
1016 / 1152 / Shouse
1017 / 1152 / Sigala
1018 / 1152 / Omi
1019 / 1152 / Koky
1020 / 1152 / Lucie
1021 / 1152 / Verona
1022 / 1152 / Anna K.
1023 / 1152 / O5
1024 / 1152 / Petr Kolář
1025 / 1152 / Mig 21
1026 / 1152 / Ego
1027 / 1152 / Kristina
1028 / 1152 / Voxel
1029 / 1152 / Lenny
1030 / 1152 / Jelen
1031 / 1152 / Udg
1032 / 1152 / Clou
1033 / 1152 / Lipo
1034 / 1152 / Sebastian
1035 / 1152 / Mirai
1036 / 1152 / Lake Malawi
1037 / 1152 / Peter Nagy
1038 / 1152 / Tina
1039 / 1152 / J.A.R.
1040 / 1152 / Calin
1041 / 1152 / Monkey Business
1042 / 1152 / Alice Springs
1043 / 1152 / Rest
1044 / 1152 / Queen
1045 / 1152 / Anna Jiřina
1046 / 1152 / Mista
1047 / 1152 / Smack
1048 / 1152 / Citron
1049 / 1152 / Milion+
1050 / 1152 / Kali
1051 / 1152 / Sima
1052 / 1152 / Olympic
1053 / 1152 / Gleb
1054 / 1152 / Hasan
1055

In [66]:
df = pd.DataFrame(final)

In [67]:
df = df[df['stazeno'] == True]

In [70]:
df.to_csv(os.path.join('wikidata','wikidata_raw.csv'), index=False)